In [68]:

import numpy as np
import utils.preprocessing 
import os
import importlib
importlib.reload(utils.preprocessing)

ratings = utils.preprocessing.load_data(os.path.abspath('../Data/MovieLens/ml-latest-small/ratings.csv'), '../Data/MovieLens/ml-latest-small/movies.csv')
y, mask = utils.preprocessing.create_rating_matrix(ratings)
y = utils.preprocessing.convert_to_implicit(y)
mask = utils.preprocessing.augment_unobserved(mask)
x, hold_back = utils.preprocessing.hold_back_ratings(y, mask)


FileNotFoundError: [Errno 2] File b'../../Data/MovieLens/ml-latest-small/ratings.csv' does not exist: b'../../Data/MovieLens/ml-latest-small/ratings.csv'

In [1]:
import pandas as pd
import numpy as np
movies= pd.read_csv('../Data/MovieLens/ml-20m/movie_mapping.csv')
ratings = pd.read_csv('../Data/MovieLens/ml-latest-small/ratings.csv')
relevant_ratings = ratings.merge(movies, how='inner', on='movieId')
print(relevant_ratings['rating'].count())

96924


96924


In [23]:
def augment_negative_sampling(rated, max_item, negative_sampling_rate = 1.5):
    nr_rated = len(rated)
    sample = np.random.randint(0, max_item, size=int(nr_rated * negative_sampling_rate)).tolist()
    return sample + rated

def group_and_transform(ratings):
    ratings = ratings[['userId', 'mId', 'rating']]
    pd_values = ratings.sort_values('userId').values.T
    user_indices = pd_values[0]
    values = np.dstack((pd_values[1], pd_values[2]))[0]
    ukeys, index = np.unique(user_indices, True)
    arrays = np.split(values, index[1:])
    df = pd.DataFrame({'userId': ukeys, 'rated': arrays})
    df['positive'] = df['rated'].map(lambda x: [y[0] for y in x if y[1] >= 3])
    return df

def split_train_test_validate_df(ratings, train_split=0.8, validation_split=0.1):
    random_nrs = np.random.rand(len(ratings))
    train_indices = random_nrs < train_split
    test_indices = (random_nrs > train_split) & (random_nrs < (train_split + validation_split))
    validate_indices = random_nrs >  (train_split + validation_split)
    train = ratings.loc[train_indices]
    test = ratings.loc[test_indices]
    validate = ratings.loc[validate_indices]
    return train, test, validate

In [24]:
nr_items = relevant_ratings['mId'].max() + 1
train_ratings, test_ratings, validate_ratings = split_train_test_validate_df(relevant_ratings)
train_user = group_and_transform(train_ratings)
train_user['rated'] = train_user['rated'].map(lambda x: augment_negative_sampling([y[0] for y in x], nr_items))
test_user = group_and_transform(test_ratings)
test_user['rated'] = test_user['rated'].map(lambda x: [y[0] for y in x])
validate_user = group_and_transform(validate_ratings)
validate_user['rated'] = validate_user['rated'].map(lambda x: [y[0] for y in x])


In [25]:
train_data = train_user.merge(validate_user, how='left', on='userId', suffixes=('', '_eval'))
train_data.loc[train_data.rated_eval.isnull()]

,userId,rated,positive,rated_eval,positive_eval
1,2.0,"[6676, 3606, 7747, 102, 2779, 9228, 5724, 9531...","[9720.0, 7810.0, 8692.0, 3153.0, 8842.0, 8345....",NaN,NaN
48,49.0,"[8134, 3260, 6564, 8967, 8512, 7004, 1748, 270...","[9088.0, 985.0, 1063.0, 502.0, 2270.0, 7815.0,...",NaN,NaN
96,97.0,"[6496, 5969, 4345, 2039, 3538, 4648, 6517, 124...","[558.0, 3875.0, 4424.0, 3535.0, 4731.0, 2885.0...",NaN,NaN
113,114.0,"[8970, 7886, 9026, 2298, 6779, 5693, 7962, 913...","[558.0, 6793.0, 2006.0, 8073.0, 5445.0, 6509.0...",NaN,NaN
123,124.0,"[9203, 4396, 5145, 5466, 7559, 3755, 8410, 351...","[1698.0, 338.0, 2941.0, 4426.0, 5299.0, 1253.0...",NaN,NaN
172,173.0,"[185, 4388, 9126, 5757, 1915, 10085, 1646, 699...","[312.0, 562.0, 502.0, 130.0, 553.0, 338.0, 300...",NaN,NaN
174,175.0,"[6899, 4336, 4821, 9313, 110, 5749, 5182, 5799...","[8283.0, 7548.0, 2014.0, 6503.0, 6096.0, 7718....",NaN,NaN
184,185.0,"[2118, 2969, 1158, 10363, 6024, 6067, 8782, 36...","[3811.0, 3486.0, 2264.0, 569.0, 346.0, 1537.0,...",NaN,NaN
191,192.0,"[4876, 2585, 10317, 6969, 8291, 1644, 3395, 64...","[147.0, 556.0, 429.0, 331.0, 276.0, 338.0, 407...",NaN,NaN
256,257.0,"[1889, 4604, 1006, 9524, 9611, 5766, 5181, 252...","[1148.0, 704.0, 2469.0, 606.0, 3532.0, 5141.0,...",NaN,NaN


In [26]:
train_user_sample = train_user.loc[np.random.rand(len(train_user)) > 0.8]
test_data = train_user_sample.merge(test_user, how='inner', on='userId', suffixes=('', '_test'))

In [51]:
test_data.loc[test_data.positive_test.apply(lambda x: len(x)) > 3 ]

,userId,rated,positive,rated_test,positive_test
1,6.0,"[4777, 8563, 7446, 1559, 4376, 7769, 7985, 142...","[6.0, 339.0, 600.0, 301.0, 30.0, 316.0, 587.0,...","[35.0, 899.0, 150.0, 650.0, 1.0, 329.0, 649.0,...","[35.0, 899.0, 150.0, 650.0, 1.0, 329.0, 649.0,..."
3,14.0,"[7433, 9353, 5333, 9744, 1932, 6831, 5965, 151...","[31.0, 3.0, 6.0, 359.0, 365.0, 527.0, 455.0, 5...","[358.0, 267.0, 102.0, 216.0, 338.0]","[358.0, 267.0, 102.0, 216.0, 338.0]"
4,19.0,"[8784, 8416, 5358, 10062, 2144, 6609, 9802, 25...","[1938.0, 397.0, 349.0, 2342.0, 157.0, 12.0, 40...","[33.0, 11.0, 3110.0, 99.0, 1362.0, 1016.0, 111...","[33.0, 1016.0, 1117.0, 1953.0, 398.0, 3231.0, ..."
5,24.0,"[8048, 6934, 4987, 5768, 4710, 9550, 9985, 688...","[5429.0, 9821.0, 4404.0, 2130.0, 238.0, 6685.0...","[8592.0, 1438.0, 7920.0, 4762.0, 6112.0, 1498....","[8592.0, 1438.0, 7920.0, 4762.0, 6112.0, 1498...."
8,30.0,"[2973, 8707, 2019, 2248, 588, 6454, 2492, 923,...","[1151.0, 8725.0, 8419.0, 7436.0, 2270.0, 9683....","[106.0, 8703.0, 10046.0, 555.0]","[106.0, 8703.0, 10046.0, 555.0]"
9,33.0,"[2731, 4874, 9302, 3274, 3403, 7986, 2124, 989...","[106.0, 1471.0, 1042.0, 421.0, 27.0, 484.0, 81...","[452.0, 2440.0, 2016.0, 250.0, 555.0, 286.0, 2...","[452.0, 2440.0, 2016.0, 250.0, 555.0, 286.0, 2..."
11,39.0,"[7156, 6931, 4804, 2034, 7501, 7189, 1029, 563...","[1063.0, 1118.0, 1390.0, 1513.0, 554.0, 1008.0...","[2270.0, 1854.0, 1147.0, 2824.0, 2440.0, 1816....","[2270.0, 1854.0, 1147.0, 2824.0, 2440.0, 1816...."
12,58.0,"[2013, 1768, 164, 3892, 1765, 6154, 3400, 2458...","[38.0, 211.0, 195.0, 280.0, 276.0, 265.0, 190....","[562.0, 332.0, 49.0, 429.0, 529.0, 319.0, 455....","[562.0, 332.0, 49.0, 429.0, 529.0, 319.0, 455...."
13,61.0,"[4252, 6699, 9868, 3202, 8177, 1509, 2450, 191...","[1452.0, 302.0, 1118.0, 5798.0, 6019.0, 251.0,...","[1070.0, 6201.0, 160.0, 529.0, 4424.0, 1075.0]","[1070.0, 6201.0, 160.0, 529.0, 4424.0, 1075.0]"
14,63.0,"[9238, 5351, 5163, 2712, 8195, 9698, 910, 1709...","[7879.0, 853.0, 8921.0, 1061.0, 6541.0, 5244.0...","[8689.0, 4441.0, 9303.0, 4705.0, 3802.0, 1814....","[8689.0, 4441.0, 9303.0, 3802.0, 1814.0, 9418...."


In [50]:
len(test_data)

122

In [44]:
test_ratings

,userId,movieId,rating,timestamp,mId
2,7,1,4.5,1106635946,0
6,19,1,4.0,965705637,0
13,43,1,5.0,848993983,0
14,44,1,3.0,869251860,0
28,82,1,2.5,1084467729,0
42,130,1,3.0,832589610,0
60,169,1,4.5,1059427918,0
68,191,1,4.0,829759809,0
73,206,1,5.0,850763267,0
87,240,1,5.0,849122434,0
